In [1]:
!pip install -U sentence-transformers
!pip3 install datasets
!pip3 install rouge_score
!pip3 install git+https://github.com/huggingface/transformers
!pip3 install sentencepiece

     |████████████████████████████████| 81kB 5.6MB/s 
     |████████████████████████████████| 2.3MB 40.7MB/s 
     |████████████████████████████████| 1.2MB 41.3MB/s 
     |████████████████████████████████| 901kB 43.0MB/s 
     |████████████████████████████████| 3.3MB 43.6MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.2.0-cp37-none-any.whl size=123339 sha256=4b33e053ec459012c5167db23a6dbd3d671e1f72deaf1952585ce8cc9892ca7d
  Stored in directory: /root/.cache/pip/wheels/0f/06/f7/faaa96fdda87462b4fd5c47b343340e9d5531ef70d0eef8242
Successfully built sentence-transformers
     |████████████████████████████████| 235kB 7.4MB/s 
     |████████████████████████████████| 245kB 11.5MB/s 
     |████████████████████████████████| 112kB 13.9MB/s 
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ct40ej7d
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-ct40ej7d
  Installing build dependencies ... d

In [2]:
import re
def finalclean(c):
  c = re.sub(r'[^A-Za-z0-9\n., %]+', " ",c)
  c=re.sub(r'Table [0-9]*',"", c)
  return c

def removeurls(text):
    result = re.sub(r"http\S+", "", text)
    result = re.sub(r"www\S+", "", result)
    return result
def removbrackets(text):
    text=re.sub("[\(\[].*?[\)\]]", "", text)
    return text
def misc_clean(text):
    d=re.sub('[a-z0-9.-]*/[a-z0-9.-]*'," ", text)
    return d

def single(text):
   # Single character removal
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)
    return text

def removelines(text):

  text = re.sub(r"\n", ' ', text)
  return text

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp "/content/drive/My Drive/allclean/processedcopy1.csv" "train.csv"

from datasets import load_dataset
import pandas as pd
train=pd.read_csv('/content/train.csv')
train.shape
#train=train[['abstract','extractedsam']]

(20725, 9)

In [5]:
train.columns

Index(['Unnamed: 0', 'title', 'abstract', 'articles', 'wordcount_articles',
       'wordcount_abstract', 'Result', 'Introduction', 'sections'],
      dtype='object')

In [6]:
train['article1']=train['Introduction']+ train['Result']

In [7]:
import re
def finalclean(c):
  c = re.sub(r'[^A-Za-z0-9\n., %]+', " ",c)
  c=re.sub(r'Table [0-9]*',"", c)
  return c

def removeurls(text):
    result = re.sub(r"http\S+", "", text)
    result = re.sub(r"www\S+", "", result)
    return result
def removbrackets(text):
    text=re.sub("[\(\[].*?[\)\]]", "", text)
    return text
def misc_clean(text):
    d=re.sub('[a-z0-9.-]*/[a-z0-9.-]*'," ", text)
    return d

def single(text):
   # Single character removal
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)
    return text

def removelines(text):

  text = re.sub(r"\n", ' ', text)
  return text

In [8]:
#val['articles'] = val['articles'].apply(lambda x: finalclean(x))
train['articles'] = train['articles'].apply(lambda x: removeurls(x))
#val['abstract'] = val['abstract'].apply(lambda x: removeurls(x))
train['articles'] = train['articles'].apply(lambda x: removbrackets(x))
#val['abstract'] = val['abstract'].apply(lambda x: removbrackets(x))
train['articles'] = train['articles'].apply(lambda x: misc_clean(x))
#val['abstract'] = val['abstract'].apply(lambda x: misc_clean(x))
train['articles'] = train['articles'].apply(lambda x: finalclean(x))
#val['abstract'] = val['abstract'].apply(lambda x: finalclean(x))
train['articles'] = train['articles'].apply(lambda x: single(x))
train['articles'] = train['articles'].apply(lambda x: removelines(x))

In [11]:
train['condition_wc']= train['articles'].apply(lambda x:get_wordcounts(x))

In [ ]:
train=train[['abstract','article1']]

In [ ]:
train['article1'].head()

In [ ]:
train=train.reset_index(drop=True)

In [10]:
def get_wordcounts(x):
    length = len(str(x).split())
    return length

In [ ]:
train['condition_wordcount']= train['article1'].apply(lambda x:get_wordcounts(x))

In [15]:
train['ccountt']= train['articles'].apply(lambda x:get_wordcounts(x))

In [12]:
train.describe()

,Unnamed: 0,wordcount_articles,wordcount_abstract,condition_wc
count,20725.000000,20725.000000,20725.000000,20725.000000
mean,6024.118118,3184.480869,199.458963,3098.269095
std,4461.517368,1207.994428,83.878854,1174.158793
min,0.000000,501.000000,2.000000,460.000000
25%,2162.000000,2313.000000,146.000000,2253.000000
50%,4812.000000,3168.000000,197.000000,3086.000000
75%,9816.000000,4114.000000,249.000000,3997.000000
max,14999.000000,5499.000000,500.000000,5403.000000


In [17]:
train.isnull().sum()

Unnamed: 0               0
title                    0
abstract                 0
articles                 0
wordcount_articles       0
wordcount_abstract       0
Result                5209
Introduction          7413
sections                37
ccountt                  0
dtype: int64

In [13]:

xy=train[train['condition_wc']>5200]

In [14]:
 xy.shape

(4716, 11)

In [ ]:
xy.columns

Index(['Unnamed: 0', 'title', 'abstract', 'articles', 'wordcount_articles',
       'wordcount_abstract', 'Result', 'Introduction', 'sections', 'ccountt'],
      dtype='object')

Experimenting with flat extraction Experimenting

In [15]:

import transformers
import transformers as ppb
from transformers import DistilBertModel, DistilBertTokenizer, BertTokenizer, BertForTokenClassification
import numpy as np

import nltk
nltk.download('punkt')
from nltk import sent_tokenize
%tensorflow_version 1.x
from keras.preprocessing.sequence import pad_sequences

from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

import json
import matplotlib.pyplot as plt
import timeit
import torch
import textwrap
wrapper = textwrap.TextWrapper(width=70)
SEED = 1234
torch.manual_seed(SEED)
device="cuda"
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)
model=model.to(device)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
TensorFlow 1.x selected.


Using TensorFlow backend.


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
def extractivesam(paragraph):
  global globe
  globe=globe+1
  print(globe)
  paragraph_split = sent_tokenize(paragraph) 
  dog=len(paragraph_split)
  input_tokens = []
  for i in paragraph_split:
    input_tokens.append(tokenizer.encode(i, add_special_tokens=True))
  temp = []
  for i in input_tokens:
  #print(len(i))
    temp.append(len(i))
  np.max(temp)  # the longest sentence in our paragraph has 88 tokens
  input_ids = pad_sequences(input_tokens, maxlen=250, dtype="long", value=0, truncating="post", padding="post")
  attention_masks = []
  for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]  # create a list of 0 and 1.
    attention_masks.append(att_mask)  # basically attention_masks is a list of list
  #input_masks = create_attention_mask(input_ids)
  print("done")
  input_ids = torch.tensor(input_ids) 
  input_ids = input_ids.to(device) 
  attention_mask = torch.tensor(attention_masks)
  attention_mask = attention_mask.to(device)

  with torch.no_grad():

    last_hidden_states = model(input_ids, attention_mask=attention_mask)
  sentence_features = last_hidden_states[0][:,0,:].detach().cpu().numpy()
  number_extract=round((0.6)*dog)
  print('performing k-medoid clustering with '
          ,number_extract,' clusters')
  print("done4")
  kmeans = KMeans(n_clusters=number_extract, 
                  random_state=0).fit(sentence_features)
  cluster_center = kmeans.cluster_centers_
  nbrs = NearestNeighbors(n_neighbors= 1, 
                          algorithm='brute').fit(sentence_features)
  distances, indices = nbrs.kneighbors(
                    cluster_center.reshape(number_extract,-1))
  print("done5")
  indices = np.sort(indices.reshape(1,-1))
  topic_answer = []
  # for i in range(len(indices)):
  #   topic_i = []
  #   for j in indices[i]:
  #     topic_i.append(paragraph_split[j])
  #   topic_answer.append(topic_i)

  for i in indices[0]:
    #print(i)
    topic_answer.append(paragraph_split[i])
  extract_num=len(topic_answer)
  #print('result:')

  #print('the ',number_extract,' extracted sentences are')
  #for i in topic_answer:
  #  print(i)
  #print(counter)
  topic_answer_string = ''
  for topic in topic_answer:
    topic_answer_string = topic_answer_string + ' '+ topic 
  return topic_answer_string

In [23]:
xy.columns

Index(['Unnamed: 0', 'title', 'abstract', 'articles', 'wordcount_articles',
       'wordcount_abstract', 'Result', 'Introduction', 'sections', 'ccountt'],
      dtype='object')

In [17]:
globe=0

In [18]:
xy=xy[['articles', 'abstract']]

In [20]:
xy.shape

(4716, 2)

In [21]:
xx=xy[1:200]

In [30]:
xx.describe()

,ct
count,199.000000
mean,2633.422111
std,249.622549
min,2004.000000
25%,2442.500000
50%,2638.000000
75%,2806.000000
max,3360.000000


In [29]:
xx.columns

Index(['articles', 'abstract', 'extractivecombo', 'ct'], dtype='object')

In [28]:
xx['ct']= xx['extractivecombo'].apply(lambda x:get_wordcounts(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
%%time
xx['extractivecombo'] = xx['articles'].apply(lambda x: extractivesam(x))

1
done
performing k-medoid clustering with  109  clusters
done4
done5
2
done
performing k-medoid clustering with  90  clusters
done4
done5
3
done
performing k-medoid clustering with  116  clusters
done4
done5
4
done
performing k-medoid clustering with  82  clusters
done4
done5
5
done
performing k-medoid clustering with  115  clusters
done4
done5
6
done
performing k-medoid clustering with  123  clusters
done4
done5
7
done
performing k-medoid clustering with  96  clusters
done4
done5
8
done
performing k-medoid clustering with  94  clusters
done4
done5
9
done
performing k-medoid clustering with  103  clusters
done4
done5
10
done
performing k-medoid clustering with  100  clusters
done4
done5
11
done
performing k-medoid clustering with  87  clusters
done4
done5
12
done
performing k-medoid clustering with  127  clusters
done4
done5
13
done
performing k-medoid clustering with  110  clusters
done4
done5
14
done
performing k-medoid clustering with  90  clusters
done4
done5
15
done
performing k-

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
xx.columns

Index(['articles', 'abstract', 'extractivecombo'], dtype='object')

In [30]:
xy=xy.reset_index(drop=True)

In [31]:
xy.shape

(4716, 3)

In [34]:
xy.isnull().sum()

articles         0
abstract         0
extractedsumm    0
dtype: int64

In [32]:
test1=xy[0:100]

In [33]:
from datasets import Dataset
test1 = Dataset.from_pandas(test1)
#val = Dataset.from_pandas(val)

In [35]:
test1

Dataset({
    features: ['articles', 'abstract', 'extractedsumm'],
    num_rows: 100
})

In [33]:
xx.columns

Index(['articles', 'abstract', 'extractivecombo', 'ct'], dtype='object')

In [36]:
from datasets import Dataset
xx= Dataset.from_pandas(xx)
#val = Dataset.from_pandas(val)

In [42]:
xx

Dataset({
    features: ['articles', 'abstract', 'extractivecombo', 'ct', '__index_level_0__'],
    num_rows: 199
})

In [31]:
% cd drive/My\ Drive/

/content/drive/My Drive


In [37]:
DATASET_NAME = "pubmed"
device = "cuda"

#MODEL_ID = f"google/bigbird-pegasus-large-{DATASET_NAME}"
MODEL_ID = f"model_C"
from transformers import AutoTokenizer, AutoModelForMaskedLM
  

from transformers import EncoderDecoderModel, BigBirdModel, BigBirdForCausalLM, BigBirdTokenizer
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
#from transformers import PegasusTokenizer, BigBirdPegasusModel
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer
import torch
model = BigBirdPegasusForConditionalGeneration.from_pretrained(MODEL_ID, gradient_checkpointing=True, use_cache=False).to(device)
tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-pubmed",use_fast=False)
#tokenizer = AutoTokenizer.from_pretrained("deepset/covid_bert_base")

In [40]:
def generate_answer(batch):
    inputs_dict = tokenizer(batch["extractivecombo"], padding="max_length", max_length=4096, return_tensors="pt", truncation=True)
    inputs_dict = {k: inputs_dict[k].to(device) for k in inputs_dict}
    predicted_abstract_ids = model.generate(**inputs_dict, max_length=256, num_beams=5, length_penalty=0.8)
    batch["predicted_abstract"] = tokenizer.decode(predicted_abstract_ids[0], skip_special_tokens=True)
    print(batch["predicted_abstract"])
    return batch

In [41]:
from datasets import load_dataset, load_metric
rouge = load_metric("rouge")

In [43]:
dataset_small = xx.select(range(100))
result_small = dataset_small.map(generate_answer)

rouge.compute(predictions=result_small["predicted_abstract"], references=result_small["abstract"])

Abstract The importance of childhood community acquired pneumonia can not be emphasized enough. In the early nineties, over 25% of the annual deaths of children in developing countries before the fifth birthday were attributable to pneumonia. In the mid nineties, over 25% of the annual deaths of children in developing countries before the fifth birthday were attributable to pneumonia. In the early nineties, over 25% of the annual deaths of children in developing countries before the fifth birthday was attributable to pneumonia. In the mid nineties, over 25% of the annual deaths of children in developing countries before the fifth birthday was attributable to pneumonia. In the early nineties, over 25% of the annual deaths of children in developing countries before the fifth birthday was attributable to pneumonia. In the early nineties, over 25% of the annual deaths of children in developing countries before the fifth birthday was attributable to pneumonia. In the mid nineties, over 25% 

{'rouge1': AggregateScore(low=Score(precision=0.44514168605230386, recall=0.318099016524355, fmeasure=0.3607519118317818), mid=Score(precision=0.47115351602828454, recall=0.33852891175522015, fmeasure=0.3791153325856292), high=Score(precision=0.49687229460493815, recall=0.3587861523635414, fmeasure=0.3976894186430214)),
 'rouge2': AggregateScore(low=Score(precision=0.1356113744260649, recall=0.09529633233820049, fmeasure=0.10784611648640799), mid=Score(precision=0.1555585012523018, recall=0.10846999707734817, fmeasure=0.12318440041115442), high=Score(precision=0.17688784981428005, recall=0.12430239147682837, fmeasure=0.14043869433795692)),
 'rougeL': AggregateScore(low=Score(precision=0.2731999244989015, recall=0.19384049034340245, fmeasure=0.21962081134109682), mid=Score(precision=0.2945889480251246, recall=0.20711122018546202, fmeasure=0.23344668191887583), high=Score(precision=0.319427421417765, recall=0.222879748835671, fmeasure=0.2494425400689304)),
 'rougeLsum': AggregateScore(lo

In [45]:

device = "cuda"

#MODEL_ID = f"google/bigbird-pegasus-large-{DATASET_NAME}"
MODEL_ID = f"model_X"


model1 = BigBirdPegasusForConditionalGeneration.from_pretrained(MODEL_ID, gradient_checkpointing=True, use_cache=False).to(device)

#tokenizer = AutoTokenizer.from_pretrained("deepset/covid_bert_base")

In [48]:
def generate_answer1(batch):
    inputs_dict = tokenizer(batch["extractivecombo"], padding="max_length", max_length=4096, return_tensors="pt", truncation=True)
    inputs_dict = {k: inputs_dict[k].to(device) for k in inputs_dict}
    predicted_abstract_ids = model1.generate(**inputs_dict, max_length=256, num_beams=5, length_penalty=0.8)
    batch["predicted_abstract"] = tokenizer.decode(predicted_abstract_ids[0], skip_special_tokens=True)
    print(batch["predicted_abstract"])
    return batch

In [49]:
dataset_small = xx.select(range(100))
result_small = dataset_small.map(generate_answer1)

rouge.compute(predictions=result_small["predicted_abstract"], references=result_small["abstract"])

Abstract The importance of childhood community acquired pneumonia can not be emphasized enough. In the early nineties, over 25% of the annual deaths of children in developing countries before the fifth birthday were attributable to pneumonia. In the early nineties, over 25% of the annual deaths of children in developing countries before the fifth birthday was attributable to pneumonia. In the early nineties, over 25% of the annual deaths of children in developing countries before the fifth birthday was attributable to pneumonia. In the early nineties, over 25% of the annual deaths of children in developing countries before the fifth birthday was attributable to pneumonia. In the late nineties, over 25% of the annual deaths of children in developing countries before the fifth birthday was attributable to pneumonia. In the early nineties, over 25% of the annual deaths of children in developing countries before the fifth birthday was attributable to pneumonia. In the early nineties, over 

{'rouge1': AggregateScore(low=Score(precision=0.45580170227417016, recall=0.31175074314683177, fmeasure=0.3594888000434748), mid=Score(precision=0.48591490404807625, recall=0.3303611962270844, fmeasure=0.3772136723297044), high=Score(precision=0.5156937437764325, recall=0.3500564146520765, fmeasure=0.39617310888790236)),
 'rouge2': AggregateScore(low=Score(precision=0.14233723585451394, recall=0.09301157341278901, fmeasure=0.10875701110807047), mid=Score(precision=0.16616278493194164, recall=0.10589865056887661, fmeasure=0.12399233886201592), high=Score(precision=0.19089382360620824, recall=0.1218258836113858, fmeasure=0.1421903506512071)),
 'rougeL': AggregateScore(low=Score(precision=0.27343347675076696, recall=0.18373588641536837, fmeasure=0.21219813726811312), mid=Score(precision=0.2990567029850141, recall=0.19738837823128913, fmeasure=0.2276822913892337), high=Score(precision=0.32936007721215743, recall=0.21314843771128303, fmeasure=0.24740930070141606)),
 'rougeLsum': AggregateSc

In [52]:
device = "cuda"

#MODEL_ID = f"google/bigbird-pegasus-large-{DATASET_NAME}"
MODEL_ID = f"BigBirdBeast"


model3 = BigBirdPegasusForConditionalGeneration.from_pretrained(MODEL_ID, gradient_checkpointing=True, use_cache=False).to(device)

#tokenizer = AutoTokenizer.from_pretrained("deepset/covid_bert_base")

In [54]:
def generate_answer3(batch):
    inputs_dict = tokenizer(batch["extractivecombo"], padding="max_length", max_length=4096, return_tensors="pt", truncation=True)
    inputs_dict = {k: inputs_dict[k].to(device) for k in inputs_dict}
    predicted_abstract_ids = model3.generate(**inputs_dict, max_length=256, num_beams=5, length_penalty=0.8)
    batch["predicted_abstract"] = tokenizer.decode(predicted_abstract_ids[0], skip_special_tokens=True)
    print(batch["predicted_abstract"])
    return batch

In [55]:
dataset_small = xx.select(range(100))
result_small = dataset_small.map(generate_answer3)

rouge.compute(predictions=result_small["predicted_abstract"], references=result_small["abstract"])

Abstract The importance of childhood community acquired pneumonia cannot be emphasized enough. The early nineties, over 25% of the annual deaths of children in developing countries before the fifth birthday was attributable to pneumonia. In the early nineties, over 25% of the annual deaths of children in developing countries before the fifth birthday was attributable to pneumonia. In the early nineties, over 25% of the annual deaths of children in developing countries before the fifth birthday was attributable to pneumonia. In the early nineties, over 25% of the annual deaths of children in developing countries before the fifth birthday was attributable to pneumonia. In the early nineties, over 25% of the annual deaths of children in developing countries before the fifth birthday was attributable to pneumonia. In the early nineties, over 25% of the annual deaths of children in developing countries before the fifth birthday was attributable to pneumonia. In the early nineties, over 25% 

{'rouge1': AggregateScore(low=Score(precision=0.466815284756956, recall=0.3067261685210198, fmeasure=0.3601744534838321), mid=Score(precision=0.496693957667576, recall=0.32825537631237306, fmeasure=0.3786024026686141), high=Score(precision=0.5256089385879309, recall=0.35162637766170735, fmeasure=0.400268725743047)),
 'rouge2': AggregateScore(low=Score(precision=0.144866622624821, recall=0.09376056063408215, fmeasure=0.10960959185407497), mid=Score(precision=0.170088877007759, recall=0.10895851395327866, fmeasure=0.12703828267425865), high=Score(precision=0.1981989560938962, recall=0.128066966651244, fmeasure=0.14652477384032625)),
 'rougeL': AggregateScore(low=Score(precision=0.27439090595367216, recall=0.17878304813799967, fmeasure=0.2093945444540134), mid=Score(precision=0.30035696229063696, recall=0.19332402055067732, fmeasure=0.2247157682585882), high=Score(precision=0.3289684046083426, recall=0.21109651618523978, fmeasure=0.24401093639444624)),
 'rougeLsum': AggregateScore(low=Sco